In [1]:
from inspect import ismethod
from IPython.display import HTML as html_print
import numpy as np

from validphys.api import API
from validphys.commondataparser import parse_commondata_new

## Old-New comparison

The next cells use `parse_commondata` and `pase_commondata_new` to compare the `CommonData` objects returned by parsing common data files in the old and new styles.

At the moment the "new commondatas" are in a hard-coded folder called `new_data` (it is hardcoded in `commondataparser.py` while the old ones are taken from whatever was installed in `share/NNPDF/data`. Eventually the new commondata will also live there.

The comparison checks that the data, uncertainties and kinematics are the same (or as close as they can be). The format of the dataframes is slightly different at the moment, but this is at the moment a desired result.

In [10]:
# Helper functions

def load_old_and_new(old_name, new_name, theoryid=200):
    """Load the old and new commondata. The theory id is needed by the validphys API when using .dataset
    but it is never used. Putting 200 as default since I guess everyone has that theory installed already.
    """
    old_cd = API.dataset(dataset_input={"dataset": old_name}, use_cuts="nocuts", theoryid=theoryid).load_commondata()

    # If this doesn't find your data, change the path inside `commondataparser.py` to wherever your data is
    # the variable is `_folder_data` at the beginning of the file.
    new_cd = parse_commondata_new(new_name)
    return old_cd, new_cd

def print_check(msg, res):
    color = "green" if res else "red"
    display(html_print(f"<text style=color:{color}>equal={res} {msg}</text>"))

def check_this(cds, method, naming=None):
    test = None
    check_result = True
    if naming is None:
        naming = method
    
    for cd in cds:
        res = getattr(cd, method)
        if ismethod(res): # since sometimes we have methods, sometimes they are attributes
            res = res()
        if test is None:
            test = res
            continue
        check_result = np.allclose(res, test)
        
    print_check(f"Testing {naming} ({method})", check_result)


def run_checks(old_cd, new_cd):
    """Run through a number of checks to ensure they contain the same information"""
    cds = [old_cd, new_cd]
    
    print(" > > Testing equality between the raw values loaded by the old and new Commondata")
    
    check_this(cds, "get_cv", naming="central_values")
    #check_this(cds, "get_kintable", naming="kinematics")
    #check_this(cds, "multiplicative_errors")
    #check_this(cds, "additive_errors")

## FT Drell-yan

These datasets can be found in the `E605` branch [#1679](https://github.com/NNPDF/nnpdf/pull/1679)

In [11]:
old_cd, new_cd = load_old_and_new("CMSTOPDIFF8TEVTTRAPNORM", "CMS_TTBAR_8TEV_LJ_DIF_YTTBAR-NORM")
#old_cd, new_cd = load_old_and_new("CMSTOPDIFF8TEVTTRAPNORM", "E605_DY_38P8GEV_PXSEC")

run_checks(old_cd, new_cd)

 > > Testing equality between the raw values loaded by the old and new Commondata


In [4]:
cd = API.dataset(dataset_input={"dataset": "CMSTOPDIFF8TEVTTRAPNORM"}, use_cuts="nocuts", theoryid=200).load_commondata()

In [13]:
new_cd.commondata_table

,process,kin1,kin2,data,stat,ADD,ADD,ADD,ADD,ADD,...,MULT,MULT,MULT,MULT,MULT,MULT,MULT,MULT,MULT,MULT
entry,,,,,,,,,,,,,,,,,,,,,
1,TTB,8000.0,-1.82,0.0607,0.0,0.000013,0.001542,0.000019,0.000175,0.001029,...,0.000425,0.000304,0.000304,0.000789,0.000243,0.000728,0.000789,0.000061,0.000668,0.000546
2,TTB,8000.0,-1.09,0.2200,0.0,0.000004,-0.000005,0.000104,-0.001034,-0.002618,...,0.001320,0.000220,0.000660,0.001320,0.000220,0.000880,0.002420,0.003740,0.001760,0.000220
3,TTB,8000.0,-0.76,0.3270,0.0,0.000003,0.000121,-0.001894,-0.001262,-0.000156,...,0.001308,0.000654,0.000654,0.000327,0.000327,0.006540,0.000654,0.004905,0.004251,0.000327
4,TTB,8000.0,-0.46,0.3730,0.0,0.000003,0.000005,-0.000643,0.004534,-0.000826,...,0.001492,0.000373,0.000373,0.001865,0.000373,0.003730,0.003357,0.001865,0.001119,0.000373
5,TTB,8000.0,-0.14,0.4270,0.0,0.000003,0.000040,0.006005,-0.000912,-0.000326,...,0.002989,0.000854,0.001708,0.003416,0.000000,0.003843,0.001281,0.000000,0.002562,0.000427
6,TTB,8000.0,0.14,0.4130,0.0,0.000003,0.000003,-0.002948,-0.004346,-0.000270,...,0.003304,0.001652,0.000826,0.003304,0.000413,0.003304,0.003304,0.003304,0.001239,0.000000
7,TTB,8000.0,0.47,0.3740,0.0,0.000003,-0.000021,-0.002387,0.003080,-0.000565,...,0.001870,0.001122,0.001122,0.001870,0.000374,0.001122,0.001870,0.003740,0.003740,0.000374
8,TTB,8000.0,0.76,0.3170,0.0,0.000003,-0.000152,0.001900,0.000401,-0.000110,...,0.001902,0.001268,0.001902,0.000634,0.000000,0.001902,0.004121,0.000317,0.000951,0.000317
9,TTB,8000.0,1.09,0.2300,0.0,0.000004,0.000024,0.000222,-0.000540,-0.001854,...,0.000920,0.000920,0.000690,0.001380,0.000000,0.001840,0.001610,0.003910,0.000460,0.000230


In [16]:
old_cd.commondata_table

,process,kin1,kin2,kin3,data,stat,ADD,MULT,ADD,MULT,...,ADD,MULT,ADD,MULT,ADD,MULT,ADD,MULT,ADD,MULT
entry,,,,,,,,,,,,,,,,,,,,,
1,HQP_YQQ,-1.82,30032.89,8000.0,0.0607,0.0,0.000013,0.020977,-0.001542,-2.540791,...,0.000728,1.2,0.000789,1.3,0.000061,0.1,0.000668,1.1,0.000546,0.9
2,HQP_YQQ,-1.09,30032.89,8000.0,0.2200,0.0,0.000004,0.001905,0.000005,0.002100,...,0.000880,0.4,0.002420,1.1,0.003740,1.7,0.001760,0.8,0.000220,0.1
3,HQP_YQQ,-0.76,30032.89,8000.0,0.3270,0.0,0.000003,0.000964,-0.000121,-0.036914,...,0.006540,2.0,0.000654,0.2,0.004905,1.5,0.004251,1.3,0.000327,0.1
4,HQP_YQQ,-0.46,30032.89,8000.0,0.3730,0.0,0.000003,0.000845,-0.000005,-0.001285,...,0.003730,1.0,0.003357,0.9,0.001865,0.5,0.001119,0.3,0.000373,0.1
5,HQP_YQQ,-0.14,30032.89,8000.0,0.4270,0.0,0.000003,0.000738,-0.000040,-0.009251,...,0.003843,0.9,0.001281,0.3,0.000000,0.0,0.002562,0.6,0.000427,0.1
6,HQP_YQQ,0.14,30032.89,8000.0,0.4130,0.0,0.000003,0.000763,-0.000003,-0.000718,...,0.003304,0.8,0.003304,0.8,0.003304,0.8,0.001239,0.3,0.000000,0.0
7,HQP_YQQ,0.47,30032.89,8000.0,0.3740,0.0,0.000003,0.000843,0.000021,0.005583,...,0.001122,0.3,0.001870,0.5,0.003740,1.0,0.003740,1.0,0.000374,0.1
8,HQP_YQQ,0.76,30032.89,8000.0,0.3170,0.0,0.000003,0.000995,0.000152,0.048020,...,0.001902,0.6,0.004121,1.3,0.000317,0.1,0.000951,0.3,0.000317,0.1
9,HQP_YQQ,1.09,30032.89,8000.0,0.2300,0.0,0.000004,0.001823,-0.000024,-0.010229,...,0.001840,0.8,0.001610,0.7,0.003910,1.7,0.000460,0.2,0.000230,0.1
